In [12]:
import pandas as pd
import time
import duckdb
import numpy as np
import os

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['Date'] = pd.to_datetime(df.Date)
df['PRA'] = df.PTS + df.REB + df.AST

con = duckdb.connect(database=':memory:')
pd.set_option('display.max_columns', None)

In [17]:
def player_influencer(player, team):
    df = pd.read_csv("../tables/2025/season_gamelogs.csv")
    frst_nm = player.split(" ")[0]
    df2 = con.execute(f"""SELECT DISTINCT game_id FROM df WHERE Player = '{player}'""").fetchdf()
    gms_plyd = len(df2.game_id.tolist())
    print(f"{player} has played {gms_plyd} games for {team}")
    
    for stat in ["PTS", "AST", "TRB"]:

        b4 = con.execute(f"""SELECT * FROM
                             (SELECT *, 
                              CASE WHEN game_id IN ({str(df2.game_id.unique().tolist()).replace("[", "").replace("]", "")}) THEN 1 ELSE 0 END as {frst_nm}_played 
                              FROM df)
                             WHERE {frst_nm}_played = 0 AND game_id LIKE '%{team}%' AND Team = '{team}'""").fetchdf()
        b4 = con.execute(f"""SELECT Player, Pos, AVG({stat}) as {stat} FROM b4 GROUP BY Player, Pos ORDER BY {stat} DESC""").fetchdf()

        aftr = con.execute(f"""SELECT * FROM
                             (SELECT *, 
                              CASE WHEN game_id IN ({str(df2.game_id.unique().tolist()).replace("[", "").replace("]", "")}) THEN 1 ELSE 0 END as {frst_nm}_played 
                              FROM df)
                             WHERE {frst_nm}_played = 1 AND game_id LIKE '%{team}%' AND Team = '{team}'""").fetchdf()
        aftr = con.execute(f"""SELECT Player, Pos, AVG({stat}) as {stat} FROM aftr GROUP BY Player, Pos ORDER BY {stat} DESC""").fetchdf()

        display(con.execute(f"""SELECT b4.Player, b4.Pos, b4.{stat} as {stat}_Wo_{frst_nm}, aftr.{stat} as {stat}_W_{frst_nm}, b4.{stat} - {stat}_W_{frst_nm} as diff FROM b4 
                               JOIN aftr ON b4.Player = aftr.Player
                               WHERE b4.Player != '{player}'
                               ORDER BY diff DESC""").fetchdf())
    
player_influencer('Trae Young', 'ATL')

Trae Young has played 5 games for ATL


,Player,Pos,PTS_Wo_Trae,PTS_W_Trae,diff
0,Vit Krejci,PG,11.090909,3.600000,7.490909
1,Dyson Daniels,SG,9.909091,5.400000,4.509091
2,Zaccharie Risacher,SF,12.727273,8.666667,4.060606
3,Keaton Wallace,None,5.700000,2.200000,3.500000
4,Onyeka Okongwu,C,17.909091,15.200000,2.709091
5,Caleb Houstan,None,1.272727,0.000000,1.272727
6,Nickeil Alexander-Walker,SG,17.300000,16.200000,1.100000
7,Jalen Johnson,SF,21.363636,20.500000,0.863636
8,N'Faly Dante,C,0.333333,0.200000,0.133333
9,Luke Kennard,SG,8.222222,8.200000,0.022222


,Player,Pos,AST_Wo_Trae,AST_W_Trae,diff
0,Dyson Daniels,SG,6.909091,3.000000,3.909091
1,Jalen Johnson,SF,7.454545,4.250000,3.204545
2,Keaton Wallace,None,3.700000,1.800000,1.900000
3,Zaccharie Risacher,SF,1.818182,0.000000,1.818182
4,Vit Krejci,PG,1.909091,0.400000,1.509091
5,Nickeil Alexander-Walker,SG,3.400000,3.200000,0.200000
6,Caleb Houstan,None,0.181818,0.000000,0.181818
7,Onyeka Okongwu,C,2.545455,2.400000,0.145455
8,N'Faly Dante,C,0.000000,0.000000,0.000000
9,Jacob Toppin,None,0.000000,0.000000,0.000000


,Player,Pos,TRB_Wo_Trae,TRB_W_Trae,diff
0,Jalen Johnson,SF,10.272727,7.000000,3.272727
1,Vit Krejci,PG,2.181818,0.800000,1.381818
2,Keaton Wallace,None,2.200000,1.000000,1.200000
3,Zaccharie Risacher,SF,2.727273,1.666667,1.060606
4,Mouhamed Gueye,PF,4.363636,3.600000,0.763636
5,Dyson Daniels,SG,5.818182,5.400000,0.418182
6,Caleb Houstan,None,0.272727,0.000000,0.272727
7,Jacob Toppin,None,0.200000,0.000000,0.200000
8,Luke Kennard,SG,2.000000,2.200000,-0.200000
9,Nickeil Alexander-Walker,SG,2.500000,2.800000,-0.300000


In [9]:
for tm in ['NOP']:
    print(tm)
    display(con.execute(f"""SELECT Date, Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                            WHERE Opp = '{tm}' AND Season = 2025 AND Pos IN ('C') AND MP >= 27
                            GROUP BY Date, Team, Player, Pos
                            ORDER BY Fpts DESC""").fetchdf())
    display(con.execute(f"""SELECT * FROM df 
                        WHERE Opp = '{tm}' AND Season = 2025 AND Pos IN ('C') AND Fpts >= 20 AND MP >= 27
                        ORDER BY Fpts DESC""").fetchdf())

NOP


,Date,Team,Player,Pos,Fpts
0,10/24/2025,SAS,Victor Wembanyama,C,48.5
1,10/29/2025,DEN,Nikola Jokic,C,38.0
2,11/08/2025,SAS,Victor Wembanyama,C,35.0
3,11/14/2025,LAL,Deandre Ayton,C,31.0
4,11/04/2025,CHO,Ryan Kalkbrenner,C,27.0
5,10/24/2025,SAS,Luke Kornet,C,26.0
6,10/31/2025,LAC,Ivica Zubac,C,26.0
7,11/12/2025,POR,Donovan Clingan,C,17.5


,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,PRA
0,2025,20251024_SAS_NOP,10/24/2025,SAS,Away,NOP,Home,Victor Wembanyama,C,1,32.30,13,23,0.565,0,3,0.000,3,4,0.75,3,8,11,2,1,9,0,6,29,28.5,-12,1,0,48.5,42
1,2025,20251029_NOP_DEN,10/29/2025,DEN,Home,NOP,Away,Nikola Jokic,C,1,27.55,10,15,0.667,1,2,0.500,0,0,0.00,2,10,12,10,2,0,2,4,21,24.3,25,1,1,38.0,43
2,2025,20251108_NOP_SAS,11/08/2025,SAS,Home,NOP,Away,Victor Wembanyama,C,1,32.45,8,19,0.421,2,9,0.222,0,0,0.00,0,18,18,3,1,3,5,2,18,12.7,5,1,0,35.0,39
3,2025,20251114_LAL_NOP,11/14/2025,LAL,Away,NOP,Home,Deandre Ayton,C,1,35.57,10,11,0.909,0,0,0.000,0,0,0.00,3,13,16,2,2,1,4,5,20,20.4,35,1,0,31.0,38
4,2025,20251104_CHO_NOP,11/04/2025,CHO,Away,NOP,Home,Ryan Kalkbrenner,C,1,30.15,4,4,1.000,0,0,0.000,2,2,1.00,6,5,11,2,1,4,2,4,10,16.1,8,1,0,27.0,23
5,2025,20251024_SAS_NOP,10/24/2025,SAS,Away,NOP,Home,Luke Kornet,C,1,34.55,6,9,0.667,0,0,0.000,2,4,0.50,9,3,12,4,0,1,0,1,14,19.6,11,1,0,26.0,30
6,2025,20251031_NOP_LAC,10/31/2025,LAC,Home,NOP,Away,Ivica Zubac,C,1,33.62,5,7,0.714,0,0,0.000,4,4,1.00,1,10,11,1,0,3,0,5,14,15.6,2,1,0,26.0,26


# Defensive Stats

In [18]:
stats = ["PTS", "AST", "REB", "STL", "BLK", "PRA", "TD"]
query_str = ", ".join([f"min({s}), max({s}), AVG({s})" for s in stats])

for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
    print(pos)
    display(con.execute(f"""SELECT Opp as Team, {query_str} FROM df 
                            WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                            GROUP BY Opp
                            ORDER BY AVG(PRA)""").fetchdf())

PG


,Team,min(PTS),max(PTS),avg(PTS),min(AST),max(AST),avg(AST),min(REB),max(REB),avg(REB),min(STL),max(STL),avg(STL),min(BLK),max(BLK),avg(BLK),min(PRA),max(PRA),avg(PRA),min(TD),max(TD),avg(TD)
0,OKC,2,25,14.272727,0,10,5.636364,2,9,5.090909,0,2,1.000000,0,1,0.181818,4,40,25.000000,0,0,0.000000
1,LAL,8,30,15.833333,3,12,6.500000,0,6,3.083333,1,7,2.166667,0,2,0.500000,14,44,25.416667,0,0,0.000000
2,HOU,10,21,15.444444,2,13,7.333333,2,7,4.333333,0,2,0.888889,0,1,0.444444,19,37,27.111111,0,0,0.000000
3,CLE,6,27,16.000000,4,11,6.818182,1,9,4.363636,0,3,1.000000,0,1,0.272727,21,43,27.181818,0,0,0.000000
4,TOR,11,31,20.333333,2,7,4.777778,0,5,3.555556,0,3,1.222222,0,2,0.555556,16,42,28.666667,0,0,0.000000
5,PHI,5,29,18.222222,2,12,6.000000,2,15,5.888889,0,3,1.333333,0,1,0.333333,11,56,30.111111,0,1,0.111111
6,LAC,8,30,20.666667,0,12,5.416667,2,6,4.166667,0,4,1.416667,0,3,0.500000,21,46,30.250000,0,0,0.000000
7,CHO,8,38,20.416667,2,9,6.166667,1,8,4.083333,0,3,1.083333,0,2,0.333333,19,51,30.666667,0,0,0.000000
8,DEN,5,42,19.200000,0,13,7.200000,0,12,4.600000,0,3,1.000000,0,1,0.100000,6,55,31.000000,0,0,0.000000
9,PHO,6,30,20.000000,1,13,6.642857,0,8,4.500000,0,3,1.428571,0,2,0.500000,11,44,31.142857,0,0,0.000000


SG


,Team,min(PTS),max(PTS),avg(PTS),min(AST),max(AST),avg(AST),min(REB),max(REB),avg(REB),min(STL),max(STL),avg(STL),min(BLK),max(BLK),avg(BLK),min(PRA),max(PRA),avg(PRA),min(TD),max(TD),avg(TD)
0,OKC,8,23,13.181818,0,4,2.363636,1,10,4.454545,0,2,0.909091,0,1,0.272727,10,30,20.000000,0,0,0.0
1,CHI,2,30,13.615385,0,8,3.384615,1,11,4.461538,0,4,1.461538,0,2,0.307692,11,41,21.461538,0,0,0.0
2,HOU,7,26,14.375000,1,6,3.125000,1,9,4.125000,0,4,1.375000,0,1,0.375000,12,34,21.625000,0,0,0.0
3,GSW,6,38,15.692308,0,9,2.923077,1,7,3.076923,0,4,1.307692,0,1,0.076923,12,45,21.692308,0,0,0.0
4,TOR,6,31,14.000000,2,8,4.071429,0,8,3.642857,0,3,0.714286,0,3,0.500000,13,43,21.714286,0,0,0.0
5,WAS,9,23,14.333333,1,8,3.000000,0,11,4.416667,0,4,1.666667,0,2,0.666667,12,33,21.750000,0,0,0.0
6,BOS,5,34,14.416667,3,7,4.333333,2,7,4.416667,0,1,0.333333,0,2,0.666667,10,44,23.166667,0,0,0.0
7,IND,5,33,16.266667,0,7,3.066667,1,9,3.866667,0,3,0.933333,0,1,0.200000,8,45,23.200000,0,0,0.0
8,DET,3,42,16.076923,1,5,2.846154,1,7,4.384615,0,4,0.692308,0,3,0.615385,6,52,23.307692,0,0,0.0
9,SAS,4,41,16.600000,0,13,3.800000,2,6,3.400000,0,3,1.000000,0,1,0.400000,8,43,23.800000,0,0,0.0


SF


,Team,min(PTS),max(PTS),avg(PTS),min(AST),max(AST),avg(AST),min(REB),max(REB),avg(REB),min(STL),max(STL),avg(STL),min(BLK),max(BLK),avg(BLK),min(PRA),max(PRA),avg(PRA),min(TD),max(TD),avg(TD)
0,MIN,5,20,13.555556,0,5,2.111111,1,8,4.777778,0,4,1.111111,0,2,0.666667,15,28,20.444444,0,0,0.000000
1,SAC,10,22,14.250000,1,3,2.000000,3,5,4.250000,1,2,1.250000,0,1,0.250000,15,29,20.500000,0,0,0.000000
2,GSW,5,31,15.000000,1,6,2.000000,2,8,5.000000,0,3,1.444444,0,2,0.222222,9,38,22.000000,0,0,0.000000
3,DEN,7,23,15.125000,0,6,2.875000,3,7,4.500000,0,4,1.750000,0,1,0.250000,13,32,22.500000,0,0,0.000000
4,CHI,2,29,15.909091,0,9,3.090909,0,9,4.454545,0,3,1.090909,0,1,0.363636,2,35,23.454545,0,0,0.000000
5,PHO,7,27,15.666667,0,7,2.888889,0,10,5.000000,0,4,1.333333,0,1,0.111111,8,42,23.555556,0,0,0.000000
6,DAL,5,34,16.400000,0,7,2.650000,2,11,5.400000,0,3,1.200000,0,3,0.600000,9,49,24.450000,0,0,0.000000
7,BOS,7,27,15.818182,1,8,3.181818,2,12,6.272727,0,2,0.818182,0,1,0.272727,13,39,25.272727,0,0,0.000000
8,CHO,3,28,18.125000,1,9,3.625000,2,8,4.125000,0,3,1.000000,0,2,0.750000,7,37,25.875000,0,0,0.000000
9,BRK,7,29,17.866667,0,8,3.800000,0,9,4.800000,0,3,0.800000,0,1,0.333333,11,40,26.466667,0,0,0.000000


PF


,Team,min(PTS),max(PTS),avg(PTS),min(AST),max(AST),avg(AST),min(REB),max(REB),avg(REB),min(STL),max(STL),avg(STL),min(BLK),max(BLK),avg(BLK),min(PRA),max(PRA),avg(PRA),min(TD),max(TD),avg(TD)
0,LAC,5,29,14.125000,0,5,2.750000,2,8,5.125000,0,3,1.250000,0,3,0.625000,11,38,22.000000,0,0,0.000000
1,NOP,7,22,14.428571,0,5,2.142857,1,11,5.428571,0,2,1.000000,0,2,0.857143,13,33,22.000000,0,0,0.000000
2,ORL,8,25,14.400000,0,4,2.500000,1,10,5.400000,0,2,0.900000,0,2,0.800000,13,39,22.300000,0,0,0.000000
3,BOS,10,28,16.600000,0,4,2.000000,2,11,6.700000,0,2,0.600000,0,2,0.400000,13,38,25.300000,0,0,0.000000
4,SAS,6,27,16.250000,0,8,3.000000,2,13,6.125000,0,3,1.000000,0,3,0.750000,13,44,25.375000,0,0,0.000000
5,GSW,6,50,17.333333,0,5,2.500000,2,12,5.750000,0,3,1.000000,0,1,0.250000,12,60,25.583333,0,0,0.000000
6,DET,4,25,15.900000,0,7,2.900000,1,14,6.800000,0,2,0.700000,0,3,1.400000,11,42,25.600000,0,0,0.000000
7,DEN,13,26,18.300000,0,8,3.200000,1,8,4.600000,0,2,1.000000,0,4,1.100000,19,39,26.100000,0,0,0.000000
8,UTA,4,29,18.100000,1,7,3.500000,2,8,5.200000,0,3,1.600000,0,2,0.800000,7,42,26.800000,0,0,0.000000
9,NYK,10,37,19.777778,0,7,2.333333,0,8,5.000000,0,3,0.888889,0,2,0.777778,12,52,27.111111,0,0,0.000000


C


,Team,min(PTS),max(PTS),avg(PTS),min(AST),max(AST),avg(AST),min(REB),max(REB),avg(REB),min(STL),max(STL),avg(STL),min(BLK),max(BLK),avg(BLK),min(PRA),max(PRA),avg(PRA),min(TD),max(TD),avg(TD)
0,LAL,2,19,10.400000,1,5,2.700000,3,14,7.600000,0,3,1.200000,0,3,1.100000,11,30,20.700000,0,0,0.000000
1,ORL,2,30,13.583333,0,8,2.333333,4,13,9.250000,0,1,0.333333,0,3,1.416667,11,50,25.166667,0,0,0.000000
2,ATL,8,18,12.000000,1,9,3.500000,4,17,9.800000,0,5,1.600000,0,2,0.500000,13,43,25.300000,0,0,0.000000
3,CHO,7,26,16.000000,0,5,2.142857,3,15,7.285714,0,2,0.857143,0,3,1.000000,13,34,25.428571,0,0,0.000000
4,OKC,10,17,12.000000,2,4,3.000000,5,18,10.500000,0,1,0.500000,0,2,1.000000,18,31,25.500000,0,0,0.000000
5,NYK,4,26,14.400000,0,7,2.600000,4,14,8.700000,0,2,0.500000,0,3,0.900000,9,35,25.700000,0,0,0.000000
6,HOU,9,25,16.800000,1,4,2.400000,6,11,7.400000,0,3,1.200000,1,5,3.000000,16,40,26.600000,0,0,0.000000
7,DEN,13,19,15.666667,0,5,2.166667,3,17,9.166667,0,4,1.333333,0,2,0.833333,18,35,27.000000,0,0,0.000000
8,CLE,9,26,15.818182,1,5,2.636364,4,20,9.545455,0,3,1.090909,0,4,1.090909,18,35,28.000000,0,0,0.000000
9,DET,6,28,16.222222,1,7,2.555556,4,17,10.333333,0,4,1.777778,0,2,0.777778,19,44,29.111111,0,0,0.000000


In [34]:
plyr = 'Stephen Curry'
opp = 'MIL'
display(con.execute(f"""SELECT * FROM df WHERE Player = '{plyr}' AND Opp = '{opp}'""").fetchdf())
display(con.execute(f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG(Pts) FROM df 
                       WHERE Player = '{plyr}' AND Opp = '{opp}' AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 3""").fetchdf())

,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,PRA
0,2023,20240306_MIL_GSW,2024-03-06,GSW,Home,MIL,Away,Stephen Curry,PG,1,30.77,10,18,0.556,6,10,0.600,3,4,0.750,2,6,8,5,0,0,2,1,29,24.3,15,0,0,28.5,42
1,2024,20250210_GSW_MIL,2025-02-10,GSW,Away,MIL,Home,Stephen Curry,PG,1,33.98,12,24,0.500,6,16,0.375,8,9,0.889,0,6,6,4,0,0,2,0,38,28.2,8,0,0,30.0,48
2,2024,20250318_MIL_GSW,2025-03-18,GSW,Home,MIL,Away,Stephen Curry,PG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0
3,2025,20251030_GSW_MIL,2025-10-30,GSW,Away,MIL,Home,Stephen Curry,PG,1,34.95,8,19,0.421,4,10,0.400,7,8,0.875,0,6,6,4,1,0,5,2,27,16.3,-6,0,0,22.5,37


,Player,avg(Pts)
0,Stephen Curry,23.5


In [46]:
display(con.execute(f"""SELECT Team, Player, MIN(MP), AVG(Fpts) AS Fpts FROM df WHERE MP BETWEEN 18 AND 22 AND Season = 2025 GROUP BY Team, Player ORDER BY Fpts DESC LIMIT 25""").fetchdf())

,Team,Player,min(MP),Fpts
0,CHI,Jalen Smith,18.57,24.000000
1,WAS,Marvin Bagley III,18.67,22.000000
2,MIN,Jaden McDaniels,20.75,21.000000
3,ORL,Goga Bitadze,18.50,21.000000
4,POR,Donovan Clingan,20.90,20.750000
5,CLE,Nae'Qwan Tomlin,21.52,19.000000
6,SAS,Luke Kornet,20.13,19.000000
7,DET,Paul Reed,19.37,19.000000
8,GSW,Trayce Jackson-Davis,18.50,18.500000
9,IND,Jay Huff,21.10,18.333333


In [64]:
import unicodedata
def fantasy_scoring(df):
    
    df = df.fillna(0)
#     df["Player"] = df["Player"].apply(lambda x: x.encode("latin1").decode("utf-8"))
#     df['Player'] = df['Player'].apply(remove_accents)

    
#     for col in df.columns.difference(['Date', 'Player', 'Team', 'Team_type', 'Opp', 'Opp_type']):
#         df[col] = np.where(df[col] == 'DNP', 0, df[col])
        
#     for col in ['MP', 'FG%', '3P%', 'FT%', 'GmSc']:
#         df[col] = df[col].astype(float)

#     for col in df.columns.difference(['Date', 'Player', 'MP', 'Team', 'Team_type', 'Opp', 'Opp_type', 
#                                       'FG%', '3P%', 'FT%', 'GmSc', '+/-']):
#         df[col] = df[col].astype(int)
    
    # Columns we care about
    stat_cols = ['PTS', 'REB', 'AST', 'STL', 'BLK']

    # Count how many stats are >= 10 for each player
    df['double_count'] = (df[stat_cols] >= 10).sum(axis=1)

    # Determine double-double or triple-double
    df['DD'] = (df['double_count'] >= 2).astype(int)
    df['TD'] = (df['double_count'] >= 3).astype(int)
    
    df['40Pts'] = (df['PTS'] >= 40).astype(int)
    df['50Pts'] = (df['PTS'] >= 40).astype(int)
    
    df['Fpts'] = (df.PTS * 0.5) + (df.AST * 1) + (df.REB * 1) + (df.STL * 2) + (df.BLK * 2) + (df.TOV * -1) + (df['3PM'] * 0.5)\
                   + (df.DD * 1) + (df.TD * 2) + (df['40Pts'] * 2) + (df['50Pts'] * 2)
    df['FptsPMin'] = df.Fpts / df.MP
    df = df.drop(columns=['double_count', '40Pts', '50Pts'])
    df = df[(df.Player != 'League Average')]
    
    return df

df_temp = pd.read_html("https://www.basketball-reference.com/leagues/NBA_2026_per_minute.html")[0]
df_temp = df_temp.drop(['Rk', 'Awards'], axis=1).rename(columns={"TRB": "REB", "3P": "3PM"})
df_temp = fantasy_scoring(df_temp)
df_temp

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3PM,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,DD,TD,Fpts,FptsPMin
0,Devin Booker,29.0,PHO,SG,14.0,14.0,501.0,9.8,19.9,0.495,2.0,5.9,0.341,7.8,14.0,0.559,0.545,6.9,7.7,0.897,1.1,3.0,4.1,7.0,0.8,0.3,4.0,2.9,28.6,0,0,24.60,0.049102
1,Franz Wagner,24.0,ORL,SF,14.0,14.0,501.0,7.6,16.5,0.461,1.8,5.0,0.357,5.8,11.5,0.506,0.515,6.3,7.5,0.829,1.9,4.5,6.3,3.9,1.1,0.4,2.1,2.5,23.3,0,0,23.65,0.047206
2,Tyrese Maxey,25.0,PHI,PG,12.0,12.0,484.0,9.4,20.8,0.454,3.4,8.2,0.418,6.0,12.6,0.476,0.536,6.2,7.1,0.884,0.3,4.2,4.5,7.0,1.3,0.7,2.2,2.4,28.6,0,0,29.30,0.060537
3,Cooper Flagg,19.0,DAL,PG,14.0,14.0,478.0,6.3,14.2,0.447,1.1,4.1,0.273,5.2,10.0,0.519,0.487,2.6,3.3,0.795,1.2,5.9,7.1,3.4,1.4,0.8,2.4,2.6,16.4,0,0,21.25,0.044456
4,DeMar DeRozan,36.0,SAC,PF,14.0,14.0,476.0,7.5,15.0,0.500,1.0,2.5,0.394,6.5,12.5,0.521,0.533,3.9,4.6,0.852,0.5,2.9,3.5,3.7,1.2,0.2,1.1,2.1,19.9,0,0,19.35,0.040651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,Myron Gardner,24.0,MIA,SF,1.0,0.0,3.0,12.0,12.0,1.000,12.0,12.0,1.000,0.0,0.0,0.000,1.500,0.0,0.0,0.000,0.0,12.0,12.0,12.0,0.0,0.0,0.0,24.0,36.0,1,1,51.00,17.000000
462,Kyle Lowry,39.0,PHI,PG,1.0,0.0,3.0,12.0,12.0,1.000,12.0,12.0,1.000,0.0,0.0,0.000,1.500,0.0,0.0,0.000,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,36.0,1,1,63.00,21.000000
463,Miles Kelly,23.0,DAL,PG,1.0,0.0,2.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0,0,-18.00,-9.000000
464,Orlando Robinson,25.0,ORL,C,1.0,0.0,2.0,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0,0,0.00,0.000000


In [78]:
display(con.execute("""SELECT * FROM df_temp WHERE MP BETWEEN 225 AND 300 ORDER BY Fpts DESC LIMIT 25""").fetchdf())

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3PM,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,DD,TD,Fpts,FptsPMin
0,LaMelo Ball,24.0,CHO,PG,7.0,7.0,227.0,8.6,20.5,0.419,3.5,10.5,0.333,5.1,10.0,0.508,0.504,4.1,5.1,0.813,1.4,6.5,7.9,10.9,1.6,0.2,3.8,3.8,24.7,1,0,33.70,0.148458
1,Kristaps Porziņģis,30.0,ATL,C,9.0,9.0,232.0,7.6,17.7,0.430,2.2,7.0,0.311,5.4,10.7,0.507,0.491,6.8,7.9,0.863,2.6,5.3,7.9,5.1,0.8,2.2,1.6,4.2,24.2,0,0,30.60,0.131897
2,Anthony Edwards,24.0,MIN,SG,9.0,9.0,273.0,10.7,22.4,0.476,4.0,9.6,0.411,6.7,12.8,0.526,0.565,7.1,8.3,0.857,1.1,4.4,5.4,4.4,1.3,0.8,2.9,2.0,32.4,0,0,29.30,0.107326
3,Andre Drummond,32.0,PHI,C,11.0,3.0,236.0,5.3,9.0,0.593,0.9,2.1,0.429,4.4,6.9,0.644,0.644,1.8,3.1,0.600,5.0,9.8,14.8,1.4,1.2,1.5,0.9,3.5,13.4,1,0,28.85,0.122246
4,Isaiah Stewart,24.0,DET,C,10.0,3.0,230.0,6.6,13.3,0.494,1.9,4.9,0.387,4.7,8.5,0.556,0.565,2.2,3.3,0.667,3.6,6.9,10.5,2.3,0.3,3.0,1.6,4.5,17.2,1,0,28.35,0.123261
5,Chet Holmgren,23.0,OKC,PF,10.0,10.0,296.0,8.3,14.4,0.576,1.9,5.0,0.390,6.3,9.4,0.675,0.644,5.0,5.7,0.872,1.8,8.1,10.0,2.2,0.9,1.6,2.6,4.6,23.5,1,0,28.30,0.095608
6,Moussa Diabaté,24.0,CHO,C,13.0,1.0,300.0,6.0,9.0,0.667,0.0,0.0,0.000,6.0,9.0,0.667,0.667,4.3,6.1,0.706,6.4,6.1,12.5,1.6,1.1,2.0,1.2,4.0,16.3,1,0,28.25,0.094167
7,Goga Bitadze,26.0,ORL,C,14.0,0.0,235.0,4.6,6.6,0.698,0.3,0.9,0.333,4.3,5.7,0.757,0.721,3.2,4.4,0.724,3.5,7.4,10.9,2.5,1.1,2.8,1.1,4.6,12.7,1,0,27.60,0.117447
8,Steven Adams,32.0,HOU,C,11.0,2.0,239.0,3.8,7.4,0.510,0.0,0.0,0.000,3.8,7.4,0.510,0.510,2.6,3.5,0.739,8.4,7.1,15.5,2.4,1.2,1.2,1.8,3.6,10.1,1,0,26.95,0.112762
9,Bam Adebayo,28.0,MIA,C,8.0,8.0,243.0,8.3,18.4,0.452,2.4,7.0,0.340,5.9,11.4,0.519,0.516,4.6,5.3,0.861,1.2,8.4,9.6,3.4,1.2,0.3,2.4,1.9,23.6,0,0,26.60,0.109465
